# Jax Level 0 Data Owner Experience - Chapter 9 - Getting started with Jax MLPs, CNNs, and RNNs

Link to the original blog post by Robert Tjarko Lange: https://roberttlange.com/posts/2020/03/blog-post-10/

## Part 0 - Initial Setup

In [8]:
# Import the necessary libraries
import syft as sy
sy.requires(">=0.8-beta")
import jax
import jax.numpy as jnp
import numpy as np

✅ The installed version of syft==0.8.1b3 matches the requirement >=0.8b0


In [9]:
# Launch the domain
node = sy.orchestra.launch(name="test-domain-1", reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

SQLite Store Path:
!open file:///var/folders/sz/hkfsnn612hq56r7cs5rd540r0000gn/T/7bca415d13ed4ec881f0d0aede098dbb.sqlite



In [10]:
!pip install torchvision

You should consider upgrading via the '/Users/antti/.pyenv/versions/3.10.4/envs/jax_1_chapter_9/bin/python3.10 -m pip install --upgrade pip' command.


In [12]:
from torchvision import datasets, transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=mnist_transform)
test_dataset = datasets.MNIST('../data', train=False, transform=mnist_transform)

train_data = train_dataset.data.numpy()
test_data = test_dataset.data.numpy()

train_labels = train_dataset.targets.numpy()
test_labels = test_dataset.targets.numpy()

In [13]:
mnist_ds = sy.Dataset(name="mnist")

split_idx = int(len(train_data) * 0.5)
train_images = sy.Asset(name="train images", data=train_data[:split_idx], mock=train_data[split_idx:])
train_labels = sy.Asset(name="train labels", data=train_labels[:split_idx], mock=train_labels[split_idx:])

split_idx = int(len(test_data) * 0.5)
test_images = sy.Asset(name="test images", data=test_data[:split_idx], mock=test_data[split_idx:])
test_labels = sy.Asset(name="test labels", data=test_labels[:split_idx], mock=test_labels[split_idx:])

mnist_ds.add_asset(train_images)
mnist_ds.add_asset(test_images)
mnist_ds.add_asset(train_labels)
mnist_ds.add_asset(test_labels)


In [14]:
domain_client.upload_dataset(mnist_ds)

  0%|          | 0/4 [00:00<?, ?it/s]

Uploading: train images


 25%|██▌       | 1/4 [00:01<00:05,  1.89s/it]

Uploading: test images


100%|██████████| 4/4 [00:02<00:00,  1.73it/s]


Uploading: train labels
Uploading: test labels


<class 'syft.service.response.SyftSuccess'>: Dataset Added

### Go to the Data Scientist Notebook for Part 1!

## Part 2 - Reviewing and Approving requests

In [46]:
# Get messages from domain
messages = domain_client.api.services.messages.get_all()
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,011c79bc56e640e197dfdb66d4ff4404,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:38:41,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,de442f39e3794e89bf8ac3489259825d,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:37:43,<<class 'syft.service.request.request.Request'...
2,syft.service.message.messages.Message,7ac4215c3a5b485881dff2b18490ad43,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:37:13,<<class 'syft.service.request.request.Request'...
3,syft.service.message.messages.Message,991252a7c7024e6fad60c9c1102962a5,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:47:05,<<class 'syft.service.request.request.Request'...
4,syft.service.message.messages.Message,4f0edcfa7bb34eb1b54cdc234413db8c,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:51:10,<<class 'syft.service.request.request.Request'...


NOTE: new code exeecution requests are inserted to the list of messages instead of appending.
As a result the indices of the old messages change which might be confusing for the user.
Perhaps messages could be sorted by creation timestamp?

In [4]:
def review_request(index: int):
    request = messages[index].link
    func = request.changes[0].link
    func_name = func.service_func_name
    print(func_name)
    print(func.raw_code)

def run_submitted_function(index: int):
    request = messages[index].link
    func = request.changes[0].link
    user_func = func.unsafe_function
    real_result = user_func()
    print(real_result)
    return real_result

def accept_request(index: int, real_result: any):
    request = messages[index].link
    request.approve()
    result = request.accept_by_depositing_result(real_result)
    print(result)

In [14]:
review_request(0)


func_grad
@sy.syft_function(input_policy=sy.ExactMatch(dummy=ptr),
                  output_policy=sy.SingleExecutionExactOutput())
def func_grad(dummy):
    # Note: using different naming conventions for numpy and jax
    # compared to the original blog post, i.e. onp => np, np => jnp.
    import numpy as np
    import jax.numpy as jnp
    from jax import random, jit, grad

    # NOTE: using the same ReLU function as in the previous example
    def ReLU(x):
        """ Rectified Linear Unit (ReLU) activation function """
        return jnp.maximum(0, x)
    
    def FiniteDiffGrad(x):
        """ Compute the finite difference derivative approx for the ReLU"""
        return np.array((ReLU(x + 1e-3) - ReLU(x - 1e-3)) / (2 * 1e-3))
    
    # NOTE: generating the random matrix again.
    # Any way to share code between syft.functions?
    key = random.PRNGKey(1)
    x = random.uniform(key, (1000, 1000))

    # Compare the Jax gradient with a finite difference approximation
    print("Ja

In [18]:
result = run_submitted_function(0)


In [19]:
accept_request(0, result)

message='Request 4fa6610a7a034987b40fb0e664830e39 changes applied'


In [ ]:
# Repeat for all the remaining requests
# review_request(1)
# result = run_submitted_function(1)
# accept_request(1, result)

### Go to the Data Scientist Notebook for Part 3!